<a href="https://colab.research.google.com/github/young-hyun-park/capston_design/blob/main/inceptionv3_multilabelclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.io import decode_image, read_file
from torchsummary import summary
from PIL import Image
import os
import re
import nibabel as nib

## Data loading

In [ ]:
zip_path = sorted(glob('/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/*'))

In [ ]:
import zipfile
for path in zip_path:
  zip_file = zipfile.ZipFile(path)
  zip_file.extractall('/content/datasets/'+path.split('/')[-1].split('.')[0])
  zip_file.close()

In [ ]:
my_glob = glob('/content/datasets/images*/*.png')
print('Number of Observations: ', len(my_glob))

Number of Observations:  76666


In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/캡스톤 디자인1/NIH_Dataframe.csv')

In [ ]:
full_img_paths = {os.path.basename(x): x for x in my_glob}
df['full_path'] = df['img_ind'].map(full_img_paths.get)

In [ ]:
df

,Cardiomegaly,Hernia,Infiltration,Nodule,Emphysema,Effusion,Atelectasis,Pleural_Thickening,Pneumothorax,Mass,Fibrosis,Consolidation,Edema,Pneumonia,img_ind,full_path
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000001_000.png,/content/datasets/images_01/00000001_000.png
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000001_001.png,/content/datasets/images_01/00000001_001.png
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000001_002.png,/content/datasets/images_01/00000001_002.png
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000003_000.png,/content/datasets/images_01/00000003_000.png
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000003_001.png,/content/datasets/images_01/00000003_001.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,00030786_007.png,/content/datasets/images_12/00030786_007.png
51755,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00030789_000.png,/content/datasets/images_12/00030789_000.png
51756,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,00030793_000.png,/content/datasets/images_12/00030793_000.png
51757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,00030795_000.png,/content/datasets/images_12/00030795_000.png


In [ ]:
df_for_col = pd.read_csv('/content/drive/Shareddrives/캡스톤 디자인1/NIH_Dataframe.csv')
target_cols = df_for_col.drop(['img_ind'], axis=1).columns.to_list()
n_classes = len(target_cols)

In [ ]:
n_classes

14

In [ ]:
from sklearn.model_selection import train_test_split
paths = df['full_path'].values
columns = df.columns[:-2]
labels = df[columns].values
(train_path, val_path ,train_labels , val_labels) = train_test_split(paths,labels,test_size = 0.2,random_state = 42)

In [ ]:
#pip install -U albumentations

In [ ]:
!pip install --upgrade albumentations

     |████████████████████████████████| 102 kB 26.6 MB/s 
     |████████████████████████████████| 47.8 MB 114 kB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
#!pip uninstall opencv-python-headless

In [ ]:
#!pip install opencv-python-headless==4.1.2.30

In [ ]:
!pip install --upgrade opencv-python

     |████████████████████████████████| 60.5 MB 98.5 MB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


## Data Loading

In [ ]:
class Train_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = np.array(Image.open(path))
        img = img/255
        img= np.stack((img,)*3, axis=-1)
        label = self.labels[idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [ ]:
class Val_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = np.array(Image.open(path))
        img = img/255
        img= np.stack((img,)*3, axis=-1)
        label = self.labels[idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [ ]:
import albumentations as A                                                                           
from albumentations.pytorch import ToTensorV2

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

data_transforms = {
    'train': A.Compose(
    [
     A.HorizontalFlip(p=0.5),
     A.VerticalFlip(p=0.5),
     ToTensorV2()
     ]
    ),
    'val': A.Compose([
                      ToTensorV2()   
    ]
                           )
}
'''
A.RandomCrop(224, 224),
     A.OneOf([
              A.HorizontalFlip(p=1),
              A.RandomRotate90(p=1),
              A.VerticalFlip(p=1)            
    ], p=1), 
'''

'\nA.RandomCrop(224, 224),\n     A.OneOf([\n              A.HorizontalFlip(p=1),\n              A.RandomRotate90(p=1),\n              A.VerticalFlip(p=1)            \n    ], p=1), \n'

## Model

In [ ]:
pip install timm

In [ ]:
import timm

In [ ]:
#timm.list_models(pretrained= True)

In [ ]:
model = timm.create_model('tf_inception_v3', pretrained = True,num_classes = 14)

In [ ]:
train_data = Train_Dataset(train_path,train_labels,transform = data_transforms['train'])
val_data = Val_Dataset(val_path,val_labels,transform =  data_transforms['val'])

In [ ]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]

# Number of classes in the dataset
num_classes = 14

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 5

In [ ]:
train_data = Train_Dataset(train_path,train_labels,transform = data_transforms['train'])
val_data = Val_Dataset(val_path,val_labels,transform =  data_transforms['val'])

In [ ]:
image_datasets = {'train' : train_data , 'val' : val_data}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score , roc_curve , accuracy_score,classification_report,multilabel_confusion_matrix,auc
import tqdm

In [ ]:
from torch.types import Device
import torch.optim as optim
from torch.optim import lr_scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = torch.load('/content/drive/Shareddrives/캡스톤 디자인1/codes/inceptionv3_multilabelclassification_6_10epoch.pt').to(device)
criterion = nn.BCEWithLogitsLoss()

# specify optimizer
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001)
sgdr_partial = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=5, eta_min=0.005 )

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, scheduler , num_epochs=25):
    since = time.time()
    train_acc_history = []
    train_loss_hist = [] 
    train_classification_report =[]
    train_confusion_matrix = []
    train_auc = []
    val_loss_hist = []
    val_acc_history = [] 
    val_classification_report =[]
    val_confusion_matrix = []
    val_auc = []
    epoch_train_label = []
    epoch_train_output = []
    epoch_val_label = []
    epoch_val_output = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            start = time.time()
            print('phase : %r' %phase)
            if phase == 'train':
                model.train() # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_acc = 0
            running_precision =0
            running_recall = 0
            running_f1_score =0 
            running_auc = 0
            # Iterate over data.
            full_label = []
            full_output = []
            full_prediction = []
            for inputs, labels in tqdm.notebook.tqdm(dataloaders[phase]):
                inputs = inputs.to(device = device, dtype = torch.float32)
                labels = labels.to(device = device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs,labels.float())
                    pred = nn.Sigmoid()(outputs)
                    full_prediction += pred.detach().cpu().numpy().tolist()
                    pred[pred <0.5] = 0
                    pred[pred >=0.5] = 1
                    if phase == 'train':
                    # backward + optimize only if in training phase
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                full_label+= labels.detach().cpu().numpy().tolist()
                full_output+= pred.detach().cpu().numpy().tolist()
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = accuracy_score(full_label,full_output)
            epoch_classification_report = classification_report(full_label,full_output,target_names = target_cols)
            epoch_multilabel_confusion_matrix = multilabel_confusion_matrix(full_label,full_output)
             
            if phase == 'train':
              train_acc_history.append(epoch_acc)
              train_loss_hist.append(epoch_loss)
              epoch_train_label.append(full_label)
              epoch_train_output.append(full_prediction)
              train_classification_report.append(epoch_classification_report)
              train_confusion_matrix.append(epoch_multilabel_confusion_matrix)
            print('\n{} Loss: {:.4f}  Time : {:.4f}'.format(phase, epoch_loss ,epoch_acc,time.time()-start))
            #,   , , epoch_precision,epoch_recall, epoch_auc, epoch_f1_score 
            print('\n--------------------------------')
            print('\n{}Epoch {}phase classification report'.format(epoch+1,phase))
            print('--------------------------------')
            print(epoch_classification_report)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                val_loss_hist.append(epoch_loss)
                epoch_val_label.append(full_label)
                epoch_val_output.append(full_prediction)
                val_classification_report.append(epoch_classification_report)
                val_confusion_matrix.append(multilabel_confusion_matrix)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    train_dict = {'Loss' : train_loss_hist , 'Accuarcy' : train_acc_history,'Classification_report' : train_classification_report,'Confusion_matrix' :train_confusion_matrix, 'train_label' : epoch_train_label , 'train_output' : epoch_train_output}
    #
    val_dict = {'Loss' :val_loss_hist , 'Accuarcy' : val_acc_history,'Classification_report' : val_classification_report , 'Confusion_matrix' :val_confusion_matrix,'val_label' : epoch_val_label,'val_output' : epoch_val_output}
    #, 
    return model, train_dict ,val_dict

In [ ]:
criterion = nn.BCEWithLogitsLoss()
import pickle
# Train and evaluate
for i in range(10):
  start = i*5+11
  end = (i+1)*5+10
  print('\n--------------------------------')
  print('|        Epoch {} ~ {}      |'.format(start, end))
  print('\n--------------------------------')
  model_ft, train_dict, val_dict  = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,sgdr_partial, num_epochs=5)
  torch.save(model_ft, '/content/drive/Shareddrives/캡스톤 디자인1/codes/inceptionv3_multilabelclassification_{}_{}epoch.pt'.format(start, end))
  with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/inceptionv3_multilabelclassification_{}_{}epoch_train_dict.pkl'.format(start, end),'wb') as fw:
    pickle.dump(train_dict, fw)
  with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/inceptionv3_multilabelclassification_{}_{}epoch_val_dict.pkl'.format(start, end),'wb') as ff:
    pickle.dump(val_dict, ff)
    
'''

confusion maxrix
[[true negative , false positive
false negative , true positive]]

[[tn , fp
fn , tp]]

'''


--------------------------------
|        Epoch 11 ~ 15      |

--------------------------------
Epoch 1/5
----------
phase : 'train'


  0%|          | 0/5176 [00:00<?, ?it/s]